# Обучение моделей с помощью Экспериментов Azure ML

__Цель лабораторной работы:__

- обучить модель машинного обучения, используя Эксперименты (Experiments) Azure ML
- зарегистрировать обученную модель.

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core.datastore import Datastore
from azureml.train.sklearn import SKLearn

from azureml.widgets import RunDetails

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Получим конфигурацию эксперимента: 

In [ ]:
%run core.py

config = get_experiment_config('lab_4B')
init_experiment(config)
experiment_dir = get_experiment_dir(config)

config

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [ ]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

## Подготовка данных для Эксперимента

### Загрузка данных

Воспользуемся данными, загружеными в Хранилище данных в [лабороторной 2B](02B-datastores-and-datasets.ipynb):

In [ ]:
data_ds = ws.datasets.get(config['core']['dataset_name'])
print(f'Used dataset {data_ds.name}: {data_ds.description}')

### Скрипт для обучения модели

Листинг скрипта обучения модели:

In [ ]:
%%writefile scripts/train_model.py


#%% Import libraries
import os
import argparse

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

from azureml.core import Run


#%% Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg_rate', default=0.01, help='Set regularization rate')
parser.add_argument('--output_dir', type=str, dest='output_dir', default='outputs', help='Set output folder')
args = parser.parse_args()

reg = args.reg_rate
output_dir = args.output_dir


#%% Get the experiment run context
run = Run.get_context()


#%% load the diabetes data (passed as an input dataset)
print('Loading Data...')
diabetes = run.input_datasets['data'].to_pandas_dataframe()


#%% Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values


#%% Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=314)


#%% Train a logistic regression model
print(f'Training a logistic regression model with regularization rate of {reg}')
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver='liblinear').fit(X_train, y_train)


#%% Сalculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)

print(f'Accuracy: {acc}')
run.log('Accuracy', np.float(acc))


#%% Сalculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])

print(f'AUC: {str(auc)}')
run.log('AUC', np.float(auc))


#%% Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))


# the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')

# the FPR and TPR
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')

run.log_image(name='ROC', plot = fig)
plt.show()


#%% Save the trained model
# note file saved in the outputs folder is automatically uploaded into experiment record
os.makedirs(output_dir, exist_ok=True)
joblib.dump(value=model, filename=f'{output_dir}/model.pkl')


run.complete()
print('Completed.')

Скопируем скрипт и содержимое директории Эксперимента:

In [ ]:
!cp scripts/train_model.py $experiment_dir
!ls $experiment_dir

## Создадим и запустим Эксперимент

In [ ]:
# Create an experiment
experiment = Experiment(workspace=ws, name=config['experiment_name'])

# Create an SKLearn estimator
estimator = SKLearn(source_directory=experiment_dir,
                    inputs=[data_ds.as_named_input('data')], 
                    entry_script='train_model.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local',
                    pip_packages=['matplotlib']
                    )


# Run the experiment and wait result
run = experiment.submit(config=estimator)

run.wait_for_completion(show_output=True)

See training details:

In [ ]:
RunDetails(run).show()

Просмотрим метрики и выходные данные:

In [ ]:
metrics = run.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

for file in run.get_file_names():
    print(file)

## Регистрируем обученную модель

Копируем путь до обученной модели (`outputs/model.pkl`), чтобы зарегистрировать ее. Обычно это последний файл:

In [ ]:
model_path = run.get_file_names()[-1]
model_path

In [ ]:
run.register_model(model_path=model_path, model_name=config['core']['ml_model_name'],
                   tags={'Lab':'4B'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Просмотрим список зарегистрированных моделей:

In [ ]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop_name}: {prop}')
        
    print('\n')

Проcмотрите зарегистрированную модель на портале Azure ML в разделе: `<your_workspace> > Models > diabetes_model`.